In [ ]:
# import collections 
# import sys
# if sys.version_info.major == 3 and sys.version_info.minor >= 10:

#     from collections.abc import MutableMapping
# else:
#     from collections import MutableMapping
from plot_functions import *
from global_file import create_params_dict
from connection_and_distillation import *
import time
from itertools import product
import warnings

warnings.filterwarnings("error")
warnings.filterwarnings(
    "ignore", message="elementwise comparison failed; returning scalar instead, "
                      "but in the future will perform elementwise comparison")


In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

def generate_network_topology(number_of_repeaters, topology='Star', distance=1):
    G = nx.Graph()

    # Add edge nodes A and B
    G.add_node('A', pos=np.array([0, 0]))
    G.add_node('B', pos=np.array([0, distance]))

    # Add repeater nodes based on the selected topology
    if topology == 'Star':
        for i in range(1, number_of_repeaters + 1):
            node_name = f'R{i}'
            G.add_node(node_name, pos=np.array([0, i * distance / (number_of_repeaters + 1)]))
            G.add_edge('A', node_name)
            G.add_edge('B', node_name)

    elif topology == 'Ring':
        for i in range(1, number_of_repeaters + 1):
            node_name = f'R{i}'
            theta = 2 * np.pi * i / number_of_repeaters
            x = distance * np.cos(theta)
            y = distance * np.sin(theta)
            G.add_node(node_name, pos=np.array([x, y]))
            G.add_edge('A', node_name)
            G.add_edge('B', node_name)

    elif topology == 'Mesh':
        for i in range(1, number_of_repeaters + 1):
            node_name = f'R{i}'
            G.add_node(node_name, pos=np.array([i * distance, 0]))
            G.add_edge('A', node_name)
            G.add_edge('B', node_name)

    elif topology == 'Tree':
        for i in range(1, number_of_repeaters + 1):
            node_name = f'R{i}'
            G.add_node(node_name, pos=np.array([0, -i * distance / (number_of_repeaters + 1)]))
            G.add_edge('A', node_name)
            G.add_edge('B', node_name)

    return G



In [ ]:

G=nx.random_internet_as_graph(100)
# Find the diameter of the graph
diameter = nx.diameter(G)

# Find all pairs of nodes that achieve the diameter
farthest_node_pairs = []
for node in G.nodes():
    eccentricity = nx.eccentricity(G, v=node)
    if eccentricity == diameter:
        farthest_node_pairs.extend([(node, other_node) for other_node in G.nodes() if nx.shortest_path_length(G, node, other_node) == diameter])

# Display the result
print("Pairs of nodes farthest from each other:", farthest_node_pairs)

In [ ]:
# # Example usage
# number_of_repeaters = 10
# topology = 'Ring'
# distance = 2
# G = generate_network_topology(number_of_repeaters, topology, distance)


pos = nx.spring_layout(G, k=0.9, seed=42)  # Adjust the value of k # Set a seed value for reproducibility
edges = G.edges()

nx.drawing.nx_agraph.write_dot(G, "network.dot")
# nx.draw(G, pos, with_labels=True, font_size=8)
import subprocess
subprocess.run(f"dot -Tsvg network.dot > network.svg", shell=True)


In [13]:
import networkx as nx
import numpy as np


def create_repeater_network(distance, n, num_paths=1):
    """
    Returns a graph G that is a non-linear network of repeaters with multiple paths from A to B.
    """
    G = nx.Graph()
    G.add_node('A', pos=np.array([0, 0]))
    G.add_node('B', pos=np.array([0, distance]))

    for _ in range(num_paths):
        G = insert_chain(G, 'A', 'B', n)

    perturbation = global_file.params.pert  # Assuming pert is defined in global_file.py
    if perturbation:
        # Add random displacements to the node positions
        for node in G.nodes():
            G.nodes[node]['pos'] += np.random.uniform(-perturbation, perturbation, size=2)

    nx.set_edge_attributes(G, values=global_file.params.number_of_fibres, name='#fibres')
    return G

# def create_repeater_chain(distance, n):
#     """
#     Returns a graph G that is a linear chain of n repeaters
#     over the given distance. pert in global_file.py dictates
#     whether there should be any random displacements
#     """
#     G = nx.Graph()
#     G.add_node('A', pos=np.array([0, 0]))
#     G.add_node('B', pos=np.array([0, distance]))
#     G.add_edge('A', 'B')
#     if n > 0:
#         G = insert_chain(G, 'A', 'B', n)
#     nx.set_edge_attributes(G, values=global_file.params.number_of_fibres, name='#fibres')
#     return G

In [9]:
filename = 'parameters/MP_params_set2_1_repeater_150km.txt'

global_file.params = create_params_dict(filename)

G = create_repeater_chain(global_file.params.distance, int(global_file.params.n))
# ,num_paths=2
# G = assign_parameters_to_nodes(G)



# with open('pkl_files/' + filename + '.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(path_dict, f)

# plot_fidelity_vs_time_plot(G, 'A', 'B', label=filename,
#                            color=None, connected=True)
# scheme1 = Scheme(('A'), None, None, None, [0.9, 0, 0, 0.1], 1, 1)
# generate_pdf_of_scheme_graph(scheme, "A", general_distillation=False)


In [ ]:
pos = nx.spring_layout(G, k=0.9, seed=42)  # Adjust the value of k # Set a seed value for reproducibility
edges = G.edges()

nx.drawing.nx_agraph.write_dot(G, "network.dot")
# nx.draw(G, pos, with_labels=True, font_size=8)
import subprocess
subprocess.run(f"dot -Tsvg network.dot > network.svg", shell=True)

In [ ]:
candidate_paths=optimise_schemes(G, global_file.params.general_distillation,
                  symmetricoptimisation=global_file.params.symmetricoptimisation)

In [ ]:
candidate_paths

In [ ]:
pos = nx.spring_layout(G, k=0.9, seed=42)  # Adjust the value of k # Set a seed value for reproducibility
edges = G.edges()

nx.drawing.nx_agraph.write_dot(G, "network.dot")
# nx.draw(G, pos, with_labels=True, font_size=8)
import subprocess
subprocess.run(f"dot -Tsvg network.dot > network.svg", shell=True)

In [ ]:
import subprocess
subprocess.run(f"dot -Tsvg network.dot > network.svg", shell=True)